In [1]:
from imp import reload
import glob
import os

import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [3]:
import utils; reload(utils)
images, labels = utils.load_data(cv2.COLOR_BGR2YCR_CB)

vehicle = 8792 samples
non-vehicle = 8968 samples


In [4]:
import utils; reload(utils)
%time hog_features = utils.get_hog_features(images)

CPU times: user 1min 2s, sys: 1.05 s, total: 1min 3s
Wall time: 1min 4s


In [6]:
from sklearn.utils import shuffle

X, y = shuffle(hog_features, labels, random_state=0)

In [13]:
# Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

from sklearn.externals import joblib
joblib.dump(scaler, './classifiers/standard_scaler.pkl')

['./classifiers/standard_scaler.pkl']

In [11]:
# train, test split
from sklearn.model_selection import train_test_split

X_train_origin, X_test, y_train_origin, y_test = \
    train_test_split(X_scaled, y, random_state=0, stratify=y)

### Linear SVM

In [12]:
# Linear SVM
import time

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC


# DataFrame to record cross validation result
df_cv_linear_svc = pd.DataFrame()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for i, (train_index, test_index) in enumerate(skf.split(X_train_origin, y_train_origin)):
    print('Start Fold {}'.format(i + 1))
    t0_fold = time.time()

    X_train, y_train = X_train_origin[train_index], y_train_origin[train_index]
    X_val, y_val = X_train_origin[test_index], y_train_origin[test_index]

    clf = LinearSVC()
    t0_fit = time.time()
    clf.fit(X_train, y_train)
    print('Fit in {}'.format(time.time() - t0_fit))

    df_cv_linear_svc.loc[i, 'acc'] = clf.score(X_train, y_train)
    df_cv_linear_svc.loc[i, 'val_acc'] = clf.score(X_val, y_val)
    
    print('Fold {} in {}'.format(i + 1, time.time() - t0_fold))

print(df_cv_linear_svc)

# Model persistence
from sklearn.externals import joblib
joblib.dump(clf, 'linear_svm.pkl')

Start Fold 1
Fit in 5.013053894042969
Fold 1 in 6.8710479736328125
Start Fold 2
Fit in 3.6093149185180664
Fold 2 in 4.4318859577178955
Start Fold 3
Fit in 12.19232988357544
Fold 3 in 12.716301918029785
Start Fold 4
Fit in 12.10954213142395
Fold 4 in 12.634108066558838
Start Fold 5
Fit in 13.80531907081604
Fold 5 in 14.34412407875061
   acc   val_acc
0  1.0  0.987617
1  1.0  0.985360
2  1.0  0.985736
3  1.0  0.987237
4  1.0  0.988359


['linear_svm.pkl']

In [14]:
# Test accuracy
%time print('test acc = ', clf.score(X_test, y_test))

test acc =  0.968918918919
CPU times: user 60.9 ms, sys: 197 ms, total: 258 ms
Wall time: 407 ms
